In [1]:
import pandas as pd
import numpy as np



file_path = 'dpe_filtre.csv'


df= pd.read_csv(file_path, sep=',', low_memory=False)



In [2]:
from IPython.core.display import display, HTML

# Modifier la largeur des cellules dans le notebook
display(HTML("""
<style>
.container {
    width: 100% 
}
</style>
"""))


In [3]:

#df_1000=df
df_1000= df.sample(n=100000, random_state=42)  # random_state pour reproductibilité
df_appartements = df_1000[df_1000['Type_bâtiment'] == 'APPARTEMENT']
#df_appartements=df_appartements[df_appartements['Conso_5_usages_é_finale'] <= 5000]


lower_bound = df_appartements['Conso_5_usages_é_finale'].quantile(0.005)
upper_bound = df_appartements['Conso_5_usages_é_finale'].quantile(0.70)


df_appartements = df_appartements[
    (df_appartements['Conso_5_usages_é_finale'] >= lower_bound) &
    (df_appartements['Conso_5_usages_é_finale'] <= upper_bound)
]





In [11]:
# Filtrer les données pour le département 31
df_departement_31 = df_appartements[df_appartements['N°_département_(BAN)'] == 31]

# Calculer la moyenne de la colonne "Conso_5_usages_é_finale"
moyenne_conso = df_departement_31['Conso_5_usages_é_finale'].mean()

print(f"La moyenne de la consommation pour le département 31 est : {moyenne_conso:.2f} kWh/an")


La moyenne de la consommation pour le département 31 est : 4078.58 kWh/an


In [4]:
print(len(df_appartements))
print(df_appartements['N°_département_(BAN)'].head()) 


57286
220998     31.0
1043998    75.0
210881     31.0
1385215    75.0
668248     56.0
Name: N°_département_(BAN), dtype: float64


In [10]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Vérifier que les données de surface habitable ne contiennent pas de valeurs nulles
df_appartements = df_appartements[df_appartements["Surface_habitable_logement"].notna()]

# Convertir les coordonnées projétées en lat/lon (WGS84)
# EPSG:2154 (Lambert 93) vers EPSG:4326 (WGS84)
gdf = gpd.GeoDataFrame(
    df_appartements,
    geometry=[Point(xy) for xy in zip(df_appartements['Coordonnée_cartographique_X_(BAN)'], df_appartements['Coordonnée_cartographique_Y_(BAN)'])],
    crs="EPSG:2154"
)

# Reprojeter en WGS84 (EPSG:4326)
gdf = gdf.to_crs(epsg=4326)

# Extraire les nouvelles colonnes Latitude et Longitude
df_appartements['Latitude'] = gdf.geometry.y
df_appartements['Longitude'] = gdf.geometry.x

# Créer l'application Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard Interactif", style={"textAlign": "center", "color": "black", "marginBottom": "20px"}),

    dcc.Tabs([
        dcc.Tab(label="Moyenne par étiquette", children=[
            html.Div([
                html.H3("Sélectionner une étiquette DPE", style={"color": "black"}),
                dcc.Dropdown(
                    id="etiquette-dropdown",
                    options=[
                        {"label": etiquette, "value": etiquette}
                        for etiquette in sorted(df_appartements["Etiquette_DPE"].dropna().unique())
                    ],
                    placeholder="Choisir une étiquette...",
                    style={"width": "50%", "margin": "10px auto"}
                ),
                html.Div(id="moyenne-etiquette-output", style={
                    "marginTop": "20px", "fontSize": "18px", "textAlign": "center", "color": "black"
                })
            ], style={"padding": "20px"})
        ]),

        dcc.Tab(label="Carte des Consommations", children=[
            html.Div([
                html.H3("Carte des consommations par localisation", style={"textAlign": "center", "color": "black"}),

                html.Div([
                    html.Div([
                        html.H4("Sélectionner la surface habitable", style={"color": "black"}),
                        dcc.RangeSlider(
                            id="surface-habitable-slider",
                            min=df_appartements["Surface_habitable_logement"].min(),
                            max=df_appartements["Surface_habitable_logement"].max(),
                            step=1,
                            marks={
                                i: str(i) for i in range(
                                    int(df_appartements["Surface_habitable_logement"].min()),
                                    int(df_appartements["Surface_habitable_logement"].max()) + 1,
                                    10
                                )
                            },
                            value=[df_appartements["Surface_habitable_logement"].min(), df_appartements["Surface_habitable_logement"].max()],
                            tooltip={"placement": "bottom", "always_visible": True},
                        )
                    ], style={"marginBottom": "20px"}),

                    html.Div([
                        html.H4("Filtrer par étiquette DPE", style={"color": "black"}),
                        dcc.Dropdown(
                            id="etiquette-dpe-filter",
                            options=[
                                {"label": etiquette, "value": etiquette}
                                for etiquette in sorted(df_appartements["Etiquette_DPE"].unique().astype(str))
                            ],
                            multi=True,
                            placeholder="Toutes les étiquettes",
                            style={"width": "100%", "marginBottom": "10px"}
                        ),

                        html.H4("Filtrer par qualité d'isolation de l'enveloppe", style={"color": "black"}),
                        dcc.Dropdown(
                            id="isolation-filter",
                            options=[
                                {"label": iso, "value": iso}
                                for iso in df_appartements["Qualité_isolation_enveloppe"].unique()
                            ],
                            multi=True,
                            placeholder="Toutes les qualités",
                            style={"width": "100%", "marginBottom": "10px"}
                        ),

                        html.H4("Filtrer par classe d'altitude", style={"color": "black"}),
                        dcc.Dropdown(
                            id="altitude-filter",
                            options=[
                                {"label": alt, "value": alt}
                                for alt in df_appartements["Classe_altitude"].dropna().unique()
                            ],
                            multi=True,
                            placeholder="Toutes les classes",
                            style={"width": "100%", "marginBottom": "10px"}
                        ),

                        html.H4("Filtrer par classe d'inertie du bâtiment", style={"color": "black"}),
                        dcc.Dropdown(
                            id="inertie-filter",
                            options=[
                                {"label": inertie, "value": inertie}
                                for inertie in sorted(df_appartements["Classe_inertie_bâtiment"].dropna().unique().astype(str))
                            ],
                            multi=True,
                            placeholder="Toutes les classes",
                            style={"width": "100%", "marginBottom": "10px"}
                        ),

                        html.H4("Filtrer par Numéro département", style={"color": "black"}),
                        dcc.Dropdown(
                            id="departement-filter",
                            options=[
                                {"label": f" {int(dept)}", "value": dept}
                                for dept in sorted(df_appartements["N°_département_(BAN)"].dropna().unique().astype(int))
                            ],
                            multi=False,
                            placeholder="Choisir un département",
                            style={"width": "100%", "marginBottom": "10px"}
                        ),
                    ], style={"marginBottom": "20px"})
                ], style={"display": "flex", "flexDirection": "column", "gap": "15px", "padding": "20px"}),

                dcc.Graph(id="carte-consommation"),
            ], style={"backgroundColor": "#F7F7FF", "padding": "20px"})
        ])
    ])
], style={"backgroundColor": "#F7F7FF", "color": "black", "fontFamily": "Arial, sans-serif"})


# Callback pour la carte
# Callback pour la moyenne par étiquette
@app.callback(
    Output("moyenne-etiquette-output", "children"),  # Met à jour le contenu du div
    [Input("etiquette-dropdown", "value")]  # Prend la valeur sélectionnée dans le menu déroulant
)
def update_moyenne_etiquette(selected_etiquette):
    if not selected_etiquette:
        return "Veuillez sélectionner une étiquette pour voir la moyenne."

    # Filtrer les données pour l'étiquette sélectionnée
    df_filtered = df_appartements[df_appartements["Etiquette_DPE"] == selected_etiquette]

    if df_filtered.empty:
        return f"Aucune donnée disponible pour l'étiquette '{selected_etiquette}'."

    # Calculer la moyenne des consommations pour l'étiquette sélectionnée
    moyenne_conso = df_filtered["Conso_5_usages_é_finale"].mean()

    # Retourner la moyenne formatée
    return f"La consommation moyenne pour l'étiquette '{selected_etiquette}' est de {moyenne_conso:.2f} kWh."


# Callback pour la carte des consommations
@app.callback(
    Output("carte-consommation", "figure"),
    [
        Input("surface-habitable-slider", "value"),
        Input("etiquette-dpe-filter", "value"),
        Input("isolation-filter", "value"),
        Input("altitude-filter", "value"),
        Input("inertie-filter", "value"),
        Input("departement-filter", "value")  # Nouveau filtre pour le département
    ]
)
def update_carte_consommation(surface_range, selected_etiquettes, selected_isolation, selected_altitude, selected_inertie, selected_departement):
    # Filtrer les données en fonction de la plage de surface habitable
    min_surface, max_surface = surface_range
    df_filtered = df_appartements[
        (df_appartements["Surface_habitable_logement"] >= min_surface) &
        (df_appartements["Surface_habitable_logement"] <= max_surface)
    ]

    # Appliquer les filtres dynamiques
    if selected_etiquettes:
        df_filtered = df_filtered[df_filtered["Etiquette_DPE"].isin(selected_etiquettes)]
    if selected_isolation:
        df_filtered = df_filtered[df_filtered["Qualité_isolation_enveloppe"].isin(selected_isolation)]
    if selected_altitude:
        df_filtered = df_filtered[df_filtered["Classe_altitude"].isin(selected_altitude)]
    if selected_inertie:
        df_filtered = df_filtered[df_filtered["Classe_inertie_bâtiment"].isin(selected_inertie)]

    # Filtrer par département
    if selected_departement:
        df_filtered = df_filtered[df_filtered["N°_département_(BAN)"] == selected_departement]

    # Calculer la latitude et la longitude moyenne du département sélectionné pour centrer la carte
    if selected_departement:
        dept_coords = df_filtered[df_filtered["N°_département_(BAN)"] == selected_departement][["Latitude", "Longitude"]].mean()
        center_lat = dept_coords["Latitude"]
        center_lon = dept_coords["Longitude"]
    else:
        # Par défaut, centrer sur toute la France
        center_lat = 46.603354
        center_lon = 1.888334
    # Exemple d'utilisation d'une échelle de couleurs allant du vert au rouge
    color_continuous_scale = [
        [0, "green"],   # Vert au début (0%)
        [0.5, "yellow"],
        [1, "red"]      # Rouge à la fin (100%)
    ]

    # Créer une carte avec les données filtrées
    fig = px.scatter_mapbox(
        df_filtered,
        lat="Latitude",
        lon="Longitude",
        size="Surface_habitable_logement",
        color="Conso_5_usages_é_finale",
        hover_name="Etiquette_DPE",
        hover_data={
            "Conso_5_usages_é_finale": True,
            "Coordonnée_cartographique_X_(BAN)": False,
            "Coordonnée_cartographique_Y_(BAN)": False
        },
        title="Carte des consommations (filtres multiples)",
        color_continuous_scale=color_continuous_scale,
    )

    # Définir les paramètres de la carte pour zoomer sur le département ou sur toute la France
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox=dict(
            zoom=8 if selected_departement else 5,  # Zoomer plus près si un département est sélectionné
            center=dict(lat=center_lat, lon=center_lon),  # Centrer sur le département ou sur la France
            pitch=0,
            bearing=0
        ),
        dragmode="zoom",
        height=1000,  # Ajuster la taille de la carte
        width=1600   # Ajuster la taille de la carte
    )

    return fig


# Lancer l'application
if __name__ == "__main__":
    app.run_server(debug=True,port=8059)


In [7]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point



# Vérifier que les données de surface habitable ne contiennent pas de valeurs nulles
df_appartements = df_appartements[df_appartements["Surface_habitable_logement"].notna()]

# Convertir les coordonnées projées en lat/lon (WGS84)
# EPSG:2154 (Lambert 93) vers EPSG:4326 (WGS84)
gdf = gpd.GeoDataFrame(
    df_appartements,
    geometry=[Point(xy) for xy in zip(df_appartements['Coordonnée_cartographique_X_(BAN)'], df_appartements['Coordonnée_cartographique_Y_(BAN)'])],
    crs="EPSG:2154"
)
gdf = gdf.to_crs(epsg=4326)

# Extraire les nouvelles colonnes Latitude et Longitude
df_appartements['Latitude'] = gdf.geometry.y
df_appartements['Longitude'] = gdf.geometry.x

# Créer l'application Dash
app = dash.Dash(__name__)

# Mise en page du tableau de bord
# Mise en page du tableau de bord
app.layout = html.Div([
    html.H1("Dashboard Interactif", style={"textAlign": "center", "color": "black", "marginBottom": "20px"}),

    dcc.Tabs([
        dcc.Tab(label="Moyenne par étiquette", children=[
            html.Div([
                html.H3("Sélectionner une étiquette DPE", style={"color": "black"}),
                dcc.Dropdown(
                    id="etiquette-dropdown",
                    options=[
                        {"label": etiquette, "value": etiquette}
                        for etiquette in sorted(df_appartements["Etiquette_DPE"].dropna().unique())
                    ],
                    placeholder="Choisir une étiquette...",
                    style={"width": "50%", "margin": "10px auto"}
                ),
                html.Div(id="moyenne-etiquette-output", style={
                    "marginTop": "20px", "fontSize": "18px", "textAlign": "center", "color": "black"
                })
            ], style={"padding": "20px"})
        ]),

        dcc.Tab(label="Carte des Consommations", children=[
            html.Div([
                html.H3("Carte des consommations par localisation", style={"textAlign": "center", "color": "black"}),

                html.Div([
                    html.Div([
                        html.H4("Sélectionner la surface habitable", style={"color": "black"}),
                        dcc.RangeSlider(
                            id="surface-habitable-slider",
                            min=df_appartements["Surface_habitable_logement"].min(),
                            max=df_appartements["Surface_habitable_logement"].max(),
                            step=1,
                            marks={
                                i: str(i) for i in range(
                                    int(df_appartements["Surface_habitable_logement"].min()),
                                    int(df_appartements["Surface_habitable_logement"].max()) + 1,
                                    10
                                )
                            },
                            value=[df_appartements["Surface_habitable_logement"].min(), df_appartements["Surface_habitable_logement"].max()],
                            tooltip={"placement": "bottom", "always_visible": True},
                        )
                    ], style={"marginBottom": "20px"}),

                    html.Div([
                        html.H4("Filtrer par étiquette DPE", style={"color": "black"}),
                        dcc.Dropdown(
                            id="etiquette-dpe-filter",
                            options=[
                                {"label": etiquette, "value": etiquette}
                                for etiquette in sorted(df_appartements["Etiquette_DPE"].unique().astype(str))
                            ],
                            multi=True,
                            placeholder="Toutes les étiquettes",
                            style={"width": "100%", "marginBottom": "10px"}
                        ),

                        html.H4("Filtrer par qualité d'isolation de l'enveloppe", style={"color": "black"}),
                        dcc.Dropdown(
                            id="isolation-filter",
                            options=[
                                {"label": iso, "value": iso}
                                for iso in df_appartements["Qualité_isolation_enveloppe"].unique()
                            ],
                            multi=True,
                            placeholder="Toutes les qualités",
                            style={"width": "100%", "marginBottom": "10px"}
                        ),

                        html.H4("Filtrer par classe d'altitude", style={"color": "black"}),
                        dcc.Dropdown(
                            id="altitude-filter",
                            options=[
                                {"label": alt, "value": alt}
                                for alt in df_appartements["Classe_altitude"].dropna().unique()
                            ],
                            multi=True,
                            placeholder="Toutes les classes",
                            style={"width": "100%", "marginBottom": "10px"}
                        ),

                        html.H4("Filtrer par classe d'inertie du bâtiment", style={"color": "black"}),
                        dcc.Dropdown(
                            id="inertie-filter",
                            options=[
                                {"label": inertie, "value": inertie}
                                for inertie in sorted(df_appartements["Classe_inertie_bâtiment"].dropna().unique().astype(str))
                            ],
                            multi=True,
                            placeholder="Toutes les classes",
                            style={"width": "100%", "marginBottom": "10px"}
                        ),

                        html.H4("Filtrer par Numéro département", style={"color": "black"}),
                        dcc.Dropdown(
                            id="departement-filter",
                            options=[
                                {"label": f" {int(dept)}", "value": dept}
                                for dept in sorted(df_appartements["N°_département_(BAN)"].dropna().unique().astype(int))
                            ],
                            multi=False,
                            placeholder="Choisir un département",
                            style={"width": "100%", "marginBottom": "10px"}
                        ),
                    ], style={"marginBottom": "20px"})
                ], style={"display": "flex", "flexDirection": "column", "gap": "15px", "padding": "20px"}),

                dcc.Graph(id="carte-consommation"),
            ], style={"backgroundColor": "#F7F7FF", "padding": "20px"})
        ])
    ])
], style={"backgroundColor": "#F7F7FF", "color": "black", "fontFamily": "Arial, sans-serif"})


# Callback pour la carte des consommations (moyenne par département)
@app.callback(
    Output("carte-consommation", "figure"),
    [
        Input("surface-habitable-slider", "value"),
        Input("etiquette-dpe-filter", "value"),
        Input("isolation-filter", "value"),
        Input("altitude-filter", "value"),
        Input("inertie-filter", "value"),
        Input("departement-filter", "value")  # Nouveau filtre pour le département
    ]
)
def update_carte_consommation_moyenne(surface_range, selected_etiquettes, selected_isolation, selected_altitude, selected_inertie, selected_departement):
    # Filtrer les données en fonction de la plage de surface habitable
    min_surface, max_surface = surface_range
    df_filtered = df_appartements[
        (df_appartements["Surface_habitable_logement"] >= min_surface) &
        (df_appartements["Surface_habitable_logement"] <= max_surface)
    ]

    # Appliquer les filtres dynamiques
    if selected_etiquettes:
        df_filtered = df_filtered[df_filtered["Etiquette_DPE"].isin(selected_etiquettes)]
    if selected_isolation:
        df_filtered = df_filtered[df_filtered["Qualité_isolation_enveloppe"].isin(selected_isolation)]
    if selected_altitude:
        df_filtered = df_filtered[df_filtered["Classe_altitude"].isin(selected_altitude)]
    if selected_inertie:
        df_filtered = df_filtered[df_filtered["Classe_inertie_bâtiment"].isin(selected_inertie)]

    # Regrouper par département et calculer les moyennes
    grouped = df_filtered.groupby("N°_département_(BAN)").agg(
        moyenne_conso=("Conso_5_usages_é_finale", "mean"),
        moyenne_surface=("Surface_habitable_logement", "mean"),
        Latitude=("Latitude", "mean"),
        Longitude=("Longitude", "mean")
    ).reset_index()

    # Filtrer par département si nécessaire
    if selected_departement:
        grouped = grouped[grouped["N°_département_(BAN)"] == selected_departement]
    
    
    color_continuous_scale = [
        [0, "green"],   # Vert au début (0%)
        [0.5, "yellow"],
        [1, "red"]      # Rouge à la fin (100%)
    ]

    # Créer une carte avec les données moyennées par département
    fig = px.scatter_mapbox(
        grouped,
        lat="Latitude",
        lon="Longitude",
        size="moyenne_surface",
        color="moyenne_conso",
        hover_name="N°_département_(BAN)",
        hover_data={
            "moyenne_conso": True,
            "moyenne_surface": True,
            "Latitude": False,
            "Longitude": False
        },
        title="Carte des consommations moyennes par département",
        color_continuous_scale=color_continuous_scale,
    )

    # Définir les paramètres de la carte
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox=dict(
            zoom=5,
            center=dict(lat=46.603354, lon=1.888334),  # Centrer sur la France
            pitch=0,
            bearing=0
        ),
        dragmode="zoom",
        height=1000,  # Ajuster la taille de la carte
        width=1600   # Ajuster la taille de la carte
    )

    return fig


# Callback pour la moyenne par étiquette
@app.callback(
    Output("moyenne-etiquette-output", "children"),  # Met à jour le contenu du div
    [Input("etiquette-dropdown", "value")]  # Prend la valeur sélectionnée dans le menu déroulant
)
def update_moyenne_etiquette(selected_etiquette):
    if not selected_etiquette:
        return "Veuillez sélectionner une étiquette pour voir la moyenne."

    # Filtrer les données pour l'étiquette sélectionnée
    df_filtered = df_appartements[df_appartements["Etiquette_DPE"] == selected_etiquette]

    if df_filtered.empty:
        return f"Aucune donnée disponible pour l'étiquette '{selected_etiquette}'."

    # Calculer la moyenne des consommations pour l'étiquette sélectionnée
    moyenne_conso = df_filtered["Conso_5_usages_é_finale"].mean()

    # Retourner la moyenne formatée
    return f"La consommation moyenne pour l'étiquette '{selected_etiquette}' est de {moyenne_conso:.2f} kWh."

# Lancer l'application
if __name__ == "__main__":
    app.run_server(debug=True,port=8054)

In [9]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Vérifier que les données de surface habitable ne contiennent pas de valeurs nulles
df_appartements = df_appartements[df_appartements["Surface_habitable_logement"].notna()]

# Convertir les coordonnées projetées en lat/lon (WGS84)
# EPSG:2154 (Lambert 93) vers EPSG:4326 (WGS84)
gdf = gpd.GeoDataFrame(
    df_appartements,
    geometry=[Point(xy) for xy in zip(df_appartements['Coordonnée_cartographique_X_(BAN)'], df_appartements['Coordonnée_cartographique_Y_(BAN)'])],
    crs="EPSG:2154"
)
gdf = gdf.to_crs(epsg=4326)

# Extraire les nouvelles colonnes Latitude et Longitude
df_appartements['Latitude'] = gdf.geometry.y
df_appartements['Longitude'] = gdf.geometry.x
df_appartements = df_appartements.dropna(subset=['Latitude', 'Longitude'])

app = dash.Dash(__name__)

# Mise en page de l'application
app.layout = html.Div([
    html.H1("Dashboard Interactif avec Filtres Dynamiques", style={"textAlign": "center"}),

    dcc.Tabs([
        dcc.Tab(label="Carte des Consommations", children=[
            html.Div([
                html.H3("Carte des consommations par localisation", style={"textAlign": "center"}),

                html.Div([
                    html.H4("Choisir la variable pour la couleur des points", style={"color": "black"}),
                    dcc.Dropdown(
                        id="color-variable",
                        options=[
                            {"label": col, "value": col}
                            for col in df_appartements.select_dtypes(include=["float64", "int64"]).columns
                        ],
                        placeholder="Choisir une variable...",
                        style={"width": "100%"},
                    ),

                    html.H4("Filtrer par surface habitable", style={"color": "black"}),
                    dcc.RangeSlider(
                        id="surface-habitable-slider",
                        min=df_appartements["Surface_habitable_logement"].min(),
                        max=df_appartements["Surface_habitable_logement"].max(),
                        step=1,
                        marks={
                            i: str(i)
                            for i in range(
                                int(df_appartements["Surface_habitable_logement"].min()),
                                int(df_appartements["Surface_habitable_logement"].max()) + 1,
                                10,
                            )
                        },
                        value=[
                            df_appartements["Surface_habitable_logement"].min(),
                            df_appartements["Surface_habitable_logement"].max(),
                        ],
                    ),

                    html.H4("Choisir la variable pour la taille des points", style={"color": "black"}),
                    dcc.Dropdown(
                        id="size-variable",
                        options=[
                            {"label": col, "value": col}
                            for col in df_appartements.select_dtypes(include=["float64", "int64"]).columns
                        ],
                        placeholder="Choisir une variable...",
                        style={"width": "100%"},
                    ),

                    html.H4("Filtrer par Numéro département", style={"color": "black"}),
                    dcc.Dropdown(
                        id="departement-filter",
                        options=[
                            {"label": f" {int(dept)}", "value": dept}
                            for dept in sorted(df_appartements["N°_département_(BAN)"].dropna().unique().astype(int))
                        ],
                        multi=False,
                        placeholder="Choisir un département",
                        style={"width": "100%", "marginBottom": "10px"}
                    ),

                    html.H4("Choisir les quantiles pour le filtrage", style={"color": "black"}),
                    dcc.RangeSlider(
                        id="quantile-slider",
                        min=0,
                        max=100,
                        step=1,
                        marks={i: f"{i}%" for i in range(0, 101, 10)},
                        value=[0.5, 70],  # Valeurs par défaut en pourcentage
                    ),
                ], style={"padding": "20px", "backgroundColor": "#EAEAEA"}),

                dcc.Graph(id="carte-consommation"),
            ], style={"padding": "20px"}),
        ])
    ])
], style={"backgroundColor": "#F7F7FF", "color": "black", "fontFamily": "Arial, sans-serif"})


# Callback pour mettre à jour la carte
@app.callback(
    Output("carte-consommation", "figure"),
    [
        Input("color-variable", "value"),
        Input("quantile-slider", "value"),
        Input("surface-habitable-slider", "value"),
        Input("size-variable", "value"),
        Input("departement-filter", "value")
    ],
)
def update_carte(color_var, quantile_range, surface_range, size_var, selected_departement):
    min_surface, max_surface = surface_range

    # Filtrage par surface habitable
    df_filtered = df_appartements[
        (df_appartements["Surface_habitable_logement"] >= min_surface) &
        (df_appartements["Surface_habitable_logement"] <= max_surface)
    ]

    # Filtrer par département
    if selected_departement:
        df_filtered = df_filtered[df_filtered["N°_département_(BAN)"] == selected_departement]

    # Calculer la latitude et la longitude moyenne du département sélectionné pour centrer la carte
    if selected_departement:
        dept_coords = df_filtered[df_filtered["N°_département_(BAN)"] == selected_departement][["Latitude", "Longitude"]].mean()
        center_lat = dept_coords["Latitude"]
        center_lon = dept_coords["Longitude"]
    else:
        # Par défaut, centrer sur toute la France
        center_lat = 46.603354
        center_lon = 1.888334

    # Filtrage selon la variable choisie (color_var) et les quantiles dynamiques
    if color_var:
        lower_percentile = quantile_range[0] / 100  # Quantile inférieur
        upper_percentile = quantile_range[1] / 100  # Quantile supérieur
        
        # Calcul des bornes des quantiles
        lower_bound = df_filtered[color_var].quantile(lower_percentile)
        upper_bound = df_filtered[color_var].quantile(upper_percentile)

        # Filtrage des données selon les quantiles
        df_filtered = df_filtered[
            (df_filtered[color_var] >= lower_bound) &
            (df_filtered[color_var] <= upper_bound)
        ]

    # Gestion des cas où aucune donnée ou variable n'est sélectionnée
    if df_filtered.empty:
        return px.scatter_mapbox(
            pd.DataFrame(columns=["Latitude", "Longitude"]),
            lat="Latitude",
            lon="Longitude",
            title="Aucune donnée à afficher. Vérifiez vos filtres.",
        )

    # Vérifier si la variable de taille est sélectionnée, sinon utiliser la surface habitable comme taille par défaut
    if not size_var:
        size_var = "Surface_habitable_logement"  # Utiliser une variable par défaut
    
    # Générer la carte avec une palette de couleurs personnalisée
    fig = px.scatter_mapbox(
        df_filtered,
        lat="Latitude",
        lon="Longitude",
        color=color_var,
        size=size_var,
        hover_name="N°_département_(BAN)",
        hover_data={"Latitude": False, "Longitude": False},
        title="Carte dynamique",
        color_continuous_scale=[
            [0, "green"],   # Vert au début (0%)
            [0.5, "yellow"], 
            [1, "red"]      # Rouge à la fin (100%)
        ],
    )

    # Mise à jour de la mise en page de la carte
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox=dict(zoom=5 if not selected_departement else 8, center=dict(lat=center_lat, lon=center_lon)),
        dragmode="zoom",
        height=1000,  # Ajuster la taille de la carte
        width=1600   # Ajuster la taille de la carte
    )

    return fig



if __name__ == "__main__":
    app.run_server(debug=True, port=8058)
